In [36]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats

## Q1. Input preparation [2 pts]
First, you need to load in the above data, provided to you as a CSV file. As the data is from UCI repository, it is already quite clean. However, some instances contain missing values (represented as ? in the CSV file) and these have to be discarded from the training set. Also, replace the `income` column with `label`, which is 1 if `income` is `>50K` and 0 otherwise.

In [101]:
def load_data(file_name):
    """ loads and processes data in the manner specified above
    Inputs:
        file_name (str): path to csv file containing data
    Outputs:
        pd.DataFrame: processed dataframe
    """
    labels=[]
    census_df = pd.read_csv(file_name)
    census_df = census_df.replace({'?': np.nan}).dropna()
    census_df=census_df.reindex()
    census_df = census_df.reset_index(drop=True)
    incomes = census_df["income"]
    for i in incomes:
        if str(i)==">50K":
            labels.append(1)
        else:
            labels.append(0)
    census_df["label"]=labels
    census_df.drop('income', axis=1, inplace=True)                             
    return census_df
    pass

# AUTOLAB_IGNORE_START
df = load_data('census.csv')
print(df.dtypes)
print(df.tail())
print(len(df))
# AUTOLAB_IGNORE_STOP

age                int64
work_class        object
final_weight       int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
label              int64
dtype: object
       age    work_class  final_weight   education  education_num  \
30157   27       Private        257302  Assoc-acdm             12   
30158   40       Private        154374     HS-grad              9   
30159   58       Private        151910     HS-grad              9   
30160   22       Private        201490     HS-grad              9   
30161   52  Self-emp-inc        287927     HS-grad              9   

           marital_status         occupation relationship   race     sex  \
30157  Married-civ-spouse       Tech-support         Wife  White  Female   
30158  Married-civ-spouse  Machine-op-inspct

Our reference code yields the following output (pay attention to the index):
```python
>>> print(df.dtypes)
age                int64
work_class        object
final_weight       int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
label              int64
dtype: object
    
>>> print(df.tail())
       age    work_class  final_weight   education  education_num  \
30157   27       Private        257302  Assoc-acdm             12   
30158   40       Private        154374     HS-grad              9   
30159   58       Private        151910     HS-grad              9   
30160   22       Private        201490     HS-grad              9   
30161   52  Self-emp-inc        287927     HS-grad              9   

           marital_status         occupation relationship   race     sex  \
30157  Married-civ-spouse       Tech-support         Wife  White  Female   
30158  Married-civ-spouse  Machine-op-inspct      Husband  White    Male   
30159             Widowed       Adm-clerical    Unmarried  White  Female   
30160       Never-married       Adm-clerical    Own-child  White    Male   
30161  Married-civ-spouse    Exec-managerial         Wife  White  Female   

       capital_gain  capital_loss  hours_per_week native_country  label  
30157             0             0              38  United-States      0  
30158             0             0              40  United-States      1  
30159             0             0              40  United-States      0  
30160             0             0              20  United-States      0  
30161         15024             0              40  United-States      1  
>>> print(len(df))
30162
```

## Overview of Naive Bayes classifier
Let $X_1, X_2, \ldots, X_k$ be the $k$ features of a dataset, with class label given by the variable $y$. A probabilistic classifier assigns the most probable class to each instance $(x_1,\ldots,x_k)$, as expressed by
$$ \hat{y} = \arg\max_y P(y\ |\ x_1,\ldots,x_k) $$

Using Bayes' theorem, the above *posterior probability* can be rewritten as
$$ P(y\ |\ x_1,\ldots,x_k) = \frac{P(y) P(x_1,\ldots,x_n\ |\ y)}{P(x_1,\ldots,x_k)} $$
where
- $P(y)$ is the prior probability of the class
- $P(x_1,\ldots,x_k\ |\ y)$ is the likelihood of data under a class
- $P(x_1,\ldots,x_k)$ is the evidence for data

Naive Bayes classifiers assume that the feature values are conditionally independent given the class label, that is,
$ P(x_1,\ldots,x_n\ |\ y) = \prod_{i=1}^{k}P(x_i\ |\ y) $. This strong assumption helps simplify the expression for posterior probability to
$$ P(y\ |\ x_1,\ldots,x_k) = \frac{P(y) \prod_{i=1}^{k}P(x_i\ |\ y)}{P(x_1,\ldots,x_k)} $$

For a given input $(x_1,\ldots,x_k)$, $P(x_1,\ldots,x_k)$ is constant. Hence, we can simplify omit the denominator replace the equality sign with proportionality as follows:
$$ P(y\ |\ x_1,\ldots,x_k) \propto P(y) \prod_{i=1}^{k}P(x_i\ |\ y) $$

Thus, the class of a new instance can be predicted as $\hat{y} = \arg\max_y P(y) \prod_{i=1}^{k}P(x_i\ |\ y)$. Here, $P(y)$ is commonly known as the **class prior** and $P(x_i\ |\ y)$ termed **feature predictor**. The rest of the assignment deals with how each of these $k+1$ probability distributions -- $P(y), P(x_1\ |\ y), \ldots, P(x_k\ |\ y)$ -- are estimated from data.


**Note**: Observe that the computation of the final expression above involve multiplication of $k+1$ probability values (which can be really low). This can lead to an underflow of numerical precision. So, it is a good practice to use a log transform of the probabilities to avoid this underflow.

** TL;DR ** Your final take away from this cell is the following expression:
$$\hat{y} = \arg\max_y \underbrace{\log P(y)}_{log-prior} + \underbrace{\sum_{i=1}^{k} \log P(x_i\ |\ y)}_{log-likelihood}$$

Each term in the sum for log-likelihood can be regarded a partial log-likelihood based on a particular feature alone.

## Feature Predictor
The beauty of a Naive Bayes classifier lies in the fact we can mix-and-match different likelihood models for each feature predictor according to the prior knowledge we have about it and these models can be varied independent of each other. For example, we might know that $P(X_i|y)$ for some continuous feature $X_i$ is normally distributed or that $P(X_i|y)$ for some categorical feature follows multinomial distribution. In such cases, we can directly plugin the pdf/pmf of these distributions in place of $P(x_i\ |\ y)$.

In this assignment, you will be using two classes of likelihood models:
- Gaussian model, for continuous real-valued features (parameterized by mean $\mu$ and variance $\sigma$)
- Categorical model, for discrete features (parameterized by $\mathbf{p} = <p_0,\ldots,p_{l-1}>$, where $l$ is the number of values taken by this categorical feature)

You need to implement a predictor class for each likelihood model. Each predictor should implement two functionalities:
- **Parameter estimation `init()`**: Learn parameters of the likelihood model using MLE (Maximum Likelihood Estimator). You need to keep track of $k$ sets of parameters, one for each class, *in the increasing order of class id, i.e., mu[i] indicates the mean of class $i$ in the Gaussian Predictor*.
- **Partial Log-Likelihood computation for *this* feature `partial_log_likelihood()`**: Use the learnt parameters to compute the probability (density/mass for continuous/categorical features) of a given feature value. Report np.log() of this value.

The parameter estimation is for the conditional distributions $P(X|Y)$. Thus, while estimating parameters for a specific class (say class 0), you will use only those data points in the training set (or rows in the input data frame) which have class label 0.

## Q2. Gaussian Feature Predictor [8pts]
The Guassian distribution is characterized by two parameters - mean $\mu$ and standard deviation $\sigma$:
$$ f_Z(z) = \frac{1}{\sqrt{2\pi}\sigma} \exp{(-\frac{(z-\mu)^2}{2\sigma^2})} $$

Given $n$ samples $z_1, \ldots, z_n$ from the above distribution, the MLE for mean and standard deviation are:
$$ \hat{\mu} = \frac{1}{n} \sum_{j=1}^{n} z_j $$

$$ \hat{\sigma} = \sqrt{\frac{1}{n} \sum_{j=1}^{n} (z_j-\hat{\mu})^2} $$

`scipy.stats.norm` would be helpful.

In [184]:
class GaussianPredictor:
    """ Feature predictor for a normally distributed real-valued, continuous feature.
        Attributes: 
            mu (array_like) : vector containing per class mean of the feature
            sigma (array_like): vector containing per class std. deviation of the feature
    """
    # feel free to define and use any more attributes, e.g., number of classes, etc
    def __init__(self, x, y) :
        """ initializes the predictor statistics (mu, sigma) for Gaussian distribution
        Inputs:
            x (array_like): feature values (continuous)
            y (arrayy_like): class labels (0,...,k-1)
        """
        y=np.array(y)
        self.k = len(np.unique(y))#2
        self.mu = np.zeros(self.k)#zeros of 2 length
        self.sigma = np.zeros(self.k)#zeros of 2 length
        x=np.array(x)
        for i in np.unique(y):
            arr = np.array([x[indx] for indx, val in enumerate(y) if val == i])
            self.mu=np.append(self.mu,np.mean(arr))
            self.sigma=np.append(self.sigma,np.std(arr))
        self.mu=self.mu[self.k:]
        self.sigma=self.sigma[self.k:]
        

    def partial_log_likelihood(self, x):
        """ log likelihood of feature values x according to each class
        Inputs:
            x (array_like): vector of feature values
        Outputs:
            (array_like): matrix of log likelihood for this feature alone
        """
        lst=[]
        x=np.array(x)
        #lst = np.zeros(len(x),self.k)
        obj = stats.norm(self.mu,self.sigma)
        for i in x:
            lst.append(np.log(obj.pdf(i)))
        return np.array(lst)
        pass

# AUTOLAB_IGNORE_START
f = GaussianPredictor(df['age'], df['label'])
print(f.mu)
print(f.sigma)
a=f.partial_log_likelihood([43,40,100,10])
print(a)
# AUTOLAB_IGNORE_STOP

[ 36.60806039  43.95911028]
[ 13.46433407  10.2689489 ]
[[ -3.63166766  -3.2524249 ]
 [ -3.55071473  -3.32238449]
 [-14.60226337 -18.13920716]
 [ -5.47164304  -8.71608989]]


Our reference code gives the following output:
```python
>>> f.mu
array([ 36.60806039  43.95911028])
>>> f.sigma
array([ 13.46433407  10.2689489 ])
>>> f.partial_log_likelihood([43,40,100,10])
array([[ -3.63166766,  -3.2524249 ],
       [ -3.55071473,  -3.32238449],
       [-14.60226337, -18.13920716],
       [ -5.47164304,  -8.71608989]])
```

## Q3. Categorical Feature Predictor [8pts]
The categorical distribution with $l$ categories $\{0,\ldots,l-1\}$ is characterized by parameters $\mathbf{p} = (p_0,\dots,p_{l-1})$:
$$ P(z; \mathbf{p}) = p_0^{[z=0]}p_1^{[z=1]}\ldots p_{l-1}^{[z=l-1]} $$

where $[z=t]$ is 1 if $z$ is $t$ and 0 otherwise.

Given $n$ samples $z_1, \ldots, z_n$ from the above distribution, the smoothed-MLE for each $p_t$ is:
$$ \hat{p_t} = \frac{n_t + \alpha}{n + l\alpha} $$

where $n_t = \sum_{j=1}^{n} [z_j=t]$, i.e., the number of times the label $t$ occurred in the sample. The smoothing is done to avoid zero-count problem (similar in spirit to $n$-gram model in NLP).

**Note:** You have to learn the number of classes and the number and value of labels from the data. We might be testing your code on a different categorical feature.

In [210]:
class CategoricalPredictor:
    """ Feature predictor for a categorical feature.
        Attributes: 
            p (dict) : dictionary of vector containing per class probability of a feature value;
                    the keys of dictionary should exactly match the values taken by this feature
    """
    # feel free to define and use any more attributes, e.g., number of classes, etc
    def __init__(self, x, y, alpha=1) :
        """ initializes the predictor statistics (p) for Categorical distribution
        Inputs:
            x (array_like): feature values (categorical)
            y (array_like): class labels (0,...,k-1)
        """
        x=np.array(x)
        y=np.array(y)
        y_lst=list(y)
        x_unq = np.unique(x)
        l=len(x_unq)
        self.p = {}
        for i in x_unq:
            lst=[]
            n=0
            num=0
            den=0
            y_sub=[]
            for indx, val in enumerate(x):
                if val == i:
                    y_sub.append(y[indx])
            for j in np.unique(y):
                n=y_lst.count(j)
                nt=y_sub.count(j)
                num=nt+alpha      
                den=n+(l*alpha)
                lst.append(num/den)
            self.p[i]=lst
            
    def partial_log_likelihood(self, x):
        """ log likelihood of feature values x according to each class
        Inputs:
            x (array_like): vector of feature values
        Outputs:
            (array_like): matrix of log likelihood for this feature
        """
        x=np.array(x)
        lst=[]
        for i in x:
            lst.append(np.log(self.p[i]))
        return np.array(lst)
        pass
# AUTOLAB_IGNORE_START
# lst1=["Male","Male","Female","Male","Female","Female","Female","Male"]
# lst2=[0,1,1,0,1,0,1,1]
f = CategoricalPredictor(df['sex'], df['label'])
#f = CategoricalPredictor(lst1, lst2)
f.p
f.partial_log_likelihood(['Male','Female','Male'])
# AUTOLAB_IGNORE_STOP

array([[-0.48243939, -0.16040634],
       [-0.96044059, -1.90917639],
       [-0.48243939, -0.16040634]])

Our reference code gives the following output:
```python
>>> f.p
{'Female': array([ 0.38272422,  0.1482024 ]),
 'Male': array([ 0.61727578,  0.8517976 ])}
>>> f.partial_log_likelihood(['Male','Female','Male'])
array([[-0.48243939 -0.16040634]
       [-0.96044059 -1.90917639]
       [-0.48243939 -0.16040634]])
```

## Q4. Putting things together [10pts]
It's time to put all the feature predictors together and do something useful! You will implement two functions in the following class.

1. **__init__()**: Compute the log prior for each class and initialize the feature predictors (based on feature type). The smoothed prior for class $t$ is given by
$$ prior(t) = \frac{n_t + \alpha}{n + k\alpha} $$
where $n_t = \sum_{j=1}^{n} [y_j=t]$, i.e., the number of times the label $t$ occurred in the sample. 

2. **predict()**: For each instance and for each class, compute the sum of log prior and partial log likelihoods for all features. Use it to predict the final class label. Break ties by predicting the class with lower id.

**Note:** Your implementation should not assume anything about the schema of the input data frame or the number of classes. The only guarantees you have are: (1) there will be a `label` column with values $0,\ldots,k-1$ for some $k$. And the datatypes of the columns will be either `object` (string, categorical) or `int64` (integer).

In [261]:
class NaiveBayesClassifier:
    """ Naive Bayes classifier for a mixture of continuous and categorical attributes.
        We use GaussianPredictor for continuous attributes and MultinomialPredictor for categorical ones.
        Attributes:
            predictor (dict): model for P(X_i|Y) for each i
            log_prior (array_like): log P(Y)
    """
    # feel free to define and use any more attributes, e.g., number of classes, etc
    def __init__(self, df, alpha=1):
        """initializes predictors for each feature and computes class prior
        Inputs:
            df (pd.DataFrame): processed dataframe, without any missing values.
        """
        df.reset_index
        self.log_prior=[]
        self.predictor={}
        y_lst=list(df.label)
        y=np.array(y_lst)
        n=len(y)
        y_unq=np.unique(y)
        k=len(y_unq)
        for j in y_unq:
            nt=y_lst.count(j)
            self.log_prior.append(np.log((nt+alpha)/(n+k*alpha)))
        for i in df:
            if(df[i].dtype==np.object):
                self.predictor[i] = CategoricalPredictor(df[i],y_lst,alpha)
            elif(df[i].dtype==np.int64):
                self.predictor[i] = GaussianPredictor(df[i],y_lst)
        pass

    def predict(self, x):
        """ predicts label for input instances from log_prior and partial_log_likelihood of feature predictors
        Inputs:
            x (pd.DataFrame): processed dataframe, without any missing values and without class label.
        Outputs:
            (array_like): array of predicted class labels (0,..,k-1)
        """
        lst=[]
        liklihood_sum=0
        for k,v in self.predictor.items():
            if(k!="label"):
                lst.append(v.partial_log_likelihood(x[k]))
        lst_np = np.array(lst)
        added_result = np.add(lst_np[0],lst_np[1])
        for i in range(len(lst_np)):
            if i>=2:
                added_result = np.add(added_result,lst_np[i])
        added = np.add(added_result,self.log_prior)
        return np.argmax(added, axis=1)
        pass

# AUTOLAB_IGNORE_START
c = NaiveBayesClassifier(df, 0)
y_pred = c.predict(df[:-1])
#c.predictor['work_class'].p
#c.log_prior
# c.predictor
# c.predictor['hours_per_week'].mu
# AUTOLAB_IGNORE_STOP

C:\Users\Anupam\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in log
C:\Users\Anupam\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in log


Our reference code gives the following output:
```python
>>> c.log_prior
array([-0.28624642, -1.39061374])
>>> c.predictor
{'age': <__main__.GaussianPredictor instance at 0x115edbcb0>,
 'capital_gain': <__main__.GaussianPredictor instance at 0x114c19320>,
 'capital_loss': <__main__.GaussianPredictor instance at 0x114c19998>,
 'education': <__main__.CategoricalPredictor instance at 0x114c04638>,
 'education_num': <__main__.GaussianPredictor instance at 0x114c04f38>,
 'final_weight': <__main__.GaussianPredictor instance at 0x114c045a8>,
 'hours_per_week': <__main__.GaussianPredictor instance at 0x114c19ef0>,
 'marital_status': <__main__.CategoricalPredictor instance at 0x114c047a0>,
 'native_country': <__main__.CategoricalPredictor instance at 0x114c19f80>,
 'occupation': <__main__.CategoricalPredictor instance at 0x114c195a8>,
 'race': <__main__.CategoricalPredictor instance at 0x114c19bd8>,
 'relationship': <__main__.CategoricalPredictor instance at 0x114c19a28>,
 'sex': <__main__.CategoricalPredictor instance at 0x114c19d40>,
 'work_class': <__main__.CategoricalPredictor instance at 0x115edbb90>}
>>> c.predictor['hours_per_week'].mu
array([ 39.34859186  45.70657965])
>>> c.predictor['hours_per_week'].sigma
array([ 11.95051037  10.73627157])
>>> c.predictor['work_class'].p
{'Federal-gov': array([ 0.02551426,  0.04861481]),
 'Local-gov': array([ 0.0643595 ,  0.08111348]),
 'Private': array([ 0.7685177,  0.6494406]),
 'Self-emp-inc': array([ 0.02092346,  0.07991476]),
 'Self-emp-not-inc': array([ 0.07879403,  0.09509856]),
 'State-gov': array([ 0.04127306,  0.04581779]),
 'Without-pay': array([ 0.00061799,  0.        ])}
>>> y_pred.shape
(30162,)
>>> y_pred
array([0, 0, 0, ..., 0, 0, 1])
```

## Q5. Evaluation - Error rate [2pts]
If a classifier makes $n_e$ errors on a data of size $n$, its error rate is $n_e/n$. Fill the following function, to evaluate your classifier.

In [270]:
def evaluate(y_hat, y):
    """ Evaluates classifier predictions
        Inputs:
            y_hat (array_like): output from classifier
            y (array_like): true class label
        Output:
            (double): error rate as defined above
    """
    y=list(y)
    y_hat=list(y_hat)
    ne=0
    n=len(y)
    for i in range(n):
        if(y_hat[i]!=y[i]):
            ne=ne+1
    return float(ne/n)
    pass

# AUTOLAB_IGNORE_START
evaluate(y_pred, df['label'])
# AUTOLAB_IGNORE_STOP

0.17240236058616804

Our implementation yields 0.17240236058616804.